In [68]:
!pip install pathway-python bokeh

ERROR: Could not find a version that satisfies the requirement pathway-python (from versions: none)
ERROR: No matching distribution found for pathway-python


In [69]:
!pip install pathway bokeh --quiet

In [70]:
# Dynamic Pricing for Urban Parking Lots - Complete Implementation
# Capstone Project: Summer Analytics 2025

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import random
from typing import Dict, List, Tuple, Optional
import math
from dataclasses import dataclass

# Install required packages (run in Colab)
# !pip install pathway-python bokeh

try:
    import pathway as pw
    from bokeh.plotting import figure, show, output_notebook
    from bokeh.models import ColumnDataSource
    from bokeh.layouts import gridplot
    from bokeh.io import push_notebook
    print("All packages imported successfully!")
except ImportError as e:
    print(f"Import error: {e}")
    print("Please install missing packages in Colab")

All packages imported successfully!


In [72]:
#=============================================================================
# STEP 1: DATA GENERATION AND SIMULATION
# =============================================================================

@dataclass
class ParkingSpace:
    """Represents a parking space with all its attributes"""
    space_id: int
    latitude: float
    longitude: float
    capacity: int
    current_occupancy: int = 0
    queue_length: int = 0
    base_price: float = 10.0
    current_price: float = 10.0

class ParkingDataGenerator:
    """Generates realistic parking data for simulation"""

    def __init__(self, num_spaces: int = 14):
        self.num_spaces = num_spaces
        self.spaces = self._initialize_spaces()
        self.time_points = self._generate_time_points()

    def _initialize_spaces(self) -> List[ParkingSpace]:
        """Initialize parking spaces with random but realistic locations"""
        spaces = []
        # Generate spaces in a city-like grid pattern
        base_lat, base_lon = 28.6139, 77.2090  # New Delhi coordinates

        for i in range(self.num_spaces):
            # Create clustered locations (some competitive areas)
            lat_offset = random.uniform(-0.05, 0.05)
            lon_offset = random.uniform(-0.05, 0.05)

            space = ParkingSpace(
                space_id=i,
                latitude=base_lat + lat_offset,
                longitude=base_lon + lon_offset,
                capacity=random.randint(20, 100)
            )
            spaces.append(space)
        return spaces

    def _generate_time_points(self) -> List[datetime]:
        """Generate time points: 8:00 AM to 4:30 PM, 30-min intervals"""
        start_time = datetime.strptime("08:00", "%H:%M")
        time_points = []
        for i in range(18):  # 18 time points
            time_points.append(start_time + timedelta(minutes=30 * i))
        return time_points

    def generate_daily_data(self, day: int) -> pd.DataFrame:
        """Generate data for one day"""
        data = []

        for time_idx, time_point in enumerate(self.time_points):
            # Time-based demand patterns
            hour = time_point.hour
            demand_multiplier = self._get_demand_multiplier(hour)

            # Special day logic
            is_special_day = random.random() < 0.1  # 10% chance

            for space in self.spaces:
                # Traffic level (0-10 scale)
                traffic_level = random.randint(1, 10)

                # Vehicle type (weighted probabilities)
                vehicle_type = np.random.choice(['car', 'bike', 'truck'],
                                              p=[0.7, 0.2, 0.1])

                # Occupancy based on demand patterns
                base_occupancy = int(space.capacity * demand_multiplier)
                occupancy = max(0, min(space.capacity,
                                     base_occupancy + random.randint(-5, 5)))

                # Queue length (more likely when occupancy is high)
                if occupancy > 0.8 * space.capacity:
                    queue_length = random.randint(0, 10)
                else:
                    queue_length = random.randint(0, 3)

                # Update space state
                space.current_occupancy = occupancy
                space.queue_length = queue_length

                data.append({
                    'day': day,
                    'time_point': time_idx,
                    'timestamp': time_point,
                    'space_id': space.space_id,
                    'latitude': space.latitude,
                    'longitude': space.longitude,
                    'capacity': space.capacity,
                    'occupancy': occupancy,
                    'queue_length': queue_length,
                    'vehicle_type': vehicle_type,
                    'traffic_level': traffic_level,
                    'is_special_day': is_special_day
                })

        return pd.DataFrame(data)

    def _get_demand_multiplier(self, hour: int) -> float:
        """Get demand multiplier based on hour of day"""
        # Peak hours: 9-11 AM and 2-4 PM
        if 9 <= hour <= 11:
            return random.uniform(0.6, 0.9)
        elif 14 <= hour <= 16:
            return random.uniform(0.5, 0.8)
        else:
            return random.uniform(0.2, 0.5)

In [73]:
# STEP 2: PRICING MODELS IMPLEMENTATION
# =============================================================================

class PricingModel:
    """Base class for pricing models"""

    def __init__(self, base_price: float = 10.0):
        self.base_price = base_price
        self.price_bounds = (0.5, 2.0)  # 0.5x to 2x base price

    def calculate_price(self, **kwargs) -> float:
        """Calculate price - to be implemented by subclasses"""
        raise NotImplementedError

    def _bound_price(self, price: float) -> float:
        """Ensure price stays within bounds"""
        min_price = self.base_price * self.price_bounds[0]
        max_price = self.base_price * self.price_bounds[1]
        return max(min_price, min(max_price, price))

class LinearPricingModel(PricingModel):
    """Model 1: Simple linear pricing based on occupancy"""

    def __init__(self, base_price: float = 10.0, alpha: float = 0.5):
        super().__init__(base_price)
        self.alpha = alpha

    def calculate_price(self, occupancy: int, capacity: int,
                       previous_price: float = None, **kwargs) -> float:
        """Linear price increase with occupancy"""
        if previous_price is None:
            previous_price = self.base_price

        occupancy_rate = occupancy / capacity
        price_adjustment = self.alpha * occupancy_rate
        new_price = previous_price + price_adjustment

        return self._bound_price(new_price)

class DemandBasedPricingModel(PricingModel):
    """Model 2: Demand-based pricing using multiple features"""

    def __init__(self, base_price: float = 10.0):
        super().__init__(base_price)
        # Coefficients for demand function
        self.alpha = 2.0    # Occupancy weight
        self.beta = 0.1     # Queue weight
        self.gamma = 0.05   # Traffic weight (negative impact)
        self.delta = 0.3    # Special day weight
        self.epsilon = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}  # Vehicle weights
        self.lambda_factor = 0.8  # Price adjustment factor

    def calculate_demand(self, occupancy: int, capacity: int, queue_length: int,
                        traffic_level: int, is_special_day: bool,
                        vehicle_type: str) -> float:
        """Calculate demand score"""
        occupancy_rate = occupancy / capacity

        demand = (self.alpha * occupancy_rate +
                 self.beta * queue_length -
                 self.gamma * traffic_level +
                 self.delta * (1 if is_special_day else 0) +
                 self.epsilon.get(vehicle_type, 1.0))

        return demand

    def calculate_price(self, occupancy: int, capacity: int, queue_length: int,
                       traffic_level: int, is_special_day: bool,
                       vehicle_type: str, **kwargs) -> float:
        """Calculate price based on demand"""
        demand = self.calculate_demand(occupancy, capacity, queue_length,
                                     traffic_level, is_special_day, vehicle_type)

        # Normalize demand (using sigmoid-like function)
        normalized_demand = 2 / (1 + math.exp(-demand)) - 1

        # Calculate price
        price = self.base_price * (1 + self.lambda_factor * normalized_demand)

        return self._bound_price(price)

class CompetitivePricingModel(DemandBasedPricingModel):
    """Model 3: Competitive pricing with geographic proximity"""

    def __init__(self, base_price: float = 10.0, proximity_threshold: float = 0.01):
        super().__init__(base_price)
        self.proximity_threshold = proximity_threshold  # ~1km in lat/lon
        self.competition_weight = 0.3

    def calculate_distance(self, lat1: float, lon1: float,
                          lat2: float, lon2: float) -> float:
        """Calculate Euclidean distance between two points"""
        return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

    def find_competitors(self, space_id: int, all_spaces_data: pd.DataFrame) -> List[Dict]:
        """Find nearby competitors"""
        current_space = all_spaces_data[all_spaces_data['space_id'] == space_id].iloc[0]
        competitors = []

        for _, other_space in all_spaces_data.iterrows():
            if other_space['space_id'] != space_id:
                distance = self.calculate_distance(
                    current_space['latitude'], current_space['longitude'],
                    other_space['latitude'], other_space['longitude']
                )

                if distance <= self.proximity_threshold:
                    competitors.append({
                        'space_id': other_space['space_id'],
                        'distance': distance,
                        'occupancy_rate': other_space['occupancy'] / other_space['capacity']
                    })

        return competitors

    def calculate_price(self, space_id: int, occupancy: int, capacity: int,
                       queue_length: int, traffic_level: int, is_special_day: bool,
                       vehicle_type: str, all_spaces_data: pd.DataFrame = None,
                       competitor_prices: Dict[int, float] = None, **kwargs) -> float:
        """Calculate competitive price"""
        # Start with demand-based price
        base_price = super().calculate_price(occupancy, capacity, queue_length,
                                           traffic_level, is_special_day, vehicle_type)

        if all_spaces_data is None or competitor_prices is None:
            return base_price

        # Find competitors
        competitors = self.find_competitors(space_id, all_spaces_data)

        if not competitors:
            return base_price

        # Calculate competitive adjustment
        competitor_avg_price = np.mean([competitor_prices.get(comp['space_id'], base_price)
                                      for comp in competitors])

        # Current occupancy rate
        occupancy_rate = occupancy / capacity

        # Competitive logic
        competitive_adjustment = 0

        if occupancy_rate > 0.8:  # High occupancy
            # Can charge more if competitors are expensive
            if competitor_avg_price > base_price:
                competitive_adjustment = self.competition_weight * (competitor_avg_price - base_price)
        else:  # Low occupancy
            # Should be competitive with nearby lots
            if competitor_avg_price < base_price:
                competitive_adjustment = -self.competition_weight * (base_price - competitor_avg_price)

        final_price = base_price + competitive_adjustment
        return self._bound_price(final_price)


In [74]:
# STEP 3: REAL-TIME SIMULATION ENGINE
# =============================================================================

class ParkingPricingSimulator:
    """Main simulation engine for real-time pricing"""

    def __init__(self, num_spaces: int = 14, num_days: int = 5):
        self.num_spaces = num_spaces
        self.num_days = num_days
        self.data_generator = ParkingDataGenerator(num_spaces)

        # Initialize pricing models
        self.models = {
            'linear': LinearPricingModel(),
            'demand_based': DemandBasedPricingModel(),
            'competitive': CompetitivePricingModel()
        }

        # Storage for results
        self.pricing_history = []
        self.current_prices = {model_name: {i: 10.0 for i in range(num_spaces)}
                              for model_name in self.models.keys()}

    def generate_dataset(self) -> pd.DataFrame:
        """Generate complete dataset for simulation"""
        all_data = []

        print("Generating dataset...")
        for day in range(self.num_days):
            daily_data = self.data_generator.generate_daily_data(day)
            all_data.append(daily_data)
            print(f"Day {day + 1}/{self.num_days} generated")

        dataset = pd.concat(all_data, ignore_index=True)
        print(f"Dataset generated: {len(dataset)} records")
        return dataset

    def simulate_real_time_pricing(self, dataset: pd.DataFrame,
                                  delay_seconds: float = 0.5) -> pd.DataFrame:
        """Simulate real-time pricing updates"""
        print("Starting real-time simulation...")

        results = []

        # Group by day and time_point for sequential processing
        for (day, time_point), group_data in dataset.groupby(['day', 'time_point']):
            print(f"Processing Day {day + 1}, Time Point {time_point + 1}")

            timestamp = group_data.iloc[0]['timestamp']

            # Calculate prices for each model
            for model_name, model in self.models.items():

                for _, row in group_data.iterrows():
                    space_id = row['space_id']

                    # Prepare parameters for pricing
                    params = {
                        'occupancy': row['occupancy'],
                        'capacity': row['capacity'],
                        'queue_length': row['queue_length'],
                        'traffic_level': row['traffic_level'],
                        'is_special_day': row['is_special_day'],
                        'vehicle_type': row['vehicle_type'],
                        'previous_price': self.current_prices[model_name][space_id]
                    }

                    # Special handling for competitive model
                    if model_name == 'competitive':
                        params['space_id'] = space_id
                        params['all_spaces_data'] = group_data
                        params['competitor_prices'] = self.current_prices['competitive'].copy()

                    # Calculate new price
                    new_price = model.calculate_price(**params)
                    self.current_prices[model_name][space_id] = new_price

                    # Store result
                    results.append({
                        'day': day,
                        'time_point': time_point,
                        'timestamp': timestamp,
                        'space_id': space_id,
                        'model': model_name,
                        'price': new_price,
                        'occupancy': row['occupancy'],
                        'capacity': row['capacity'],
                        'occupancy_rate': row['occupancy'] / row['capacity'],
                        'queue_length': row['queue_length'],
                        'traffic_level': row['traffic_level'],
                        'is_special_day': row['is_special_day'],
                        'vehicle_type': row['vehicle_type']
                    })

            # Simulate real-time delay
            time.sleep(delay_seconds)

        return pd.DataFrame(results)

In [76]:
 #STEP 4: VISUALIZATION AND ANALYSIS
# =============================================================================

class PricingVisualizer:
    """Handles all visualization requirements"""

    def __init__(self):
        self.colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']

    def create_price_comparison_plot(self, results_df: pd.DataFrame,
                                   space_ids: List[int] = None):
        """Create price comparison plots for different models"""
        if space_ids is None:
            space_ids = results_df['space_id'].unique()[:6]  # Show first 6 spaces

        plots = []

        for i, space_id in enumerate(space_ids):
            space_data = results_df[results_df['space_id'] == space_id]

            p = figure(title=f'Parking Space {space_id} - Dynamic Pricing',
                      x_axis_label='Time Point', y_axis_label='Price ($)',
                      width=400, height=300)

            for model_name in ['linear', 'demand_based', 'competitive']:
                model_data = space_data[space_data['model'] == model_name]
                if not model_data.empty:
                    p.line(model_data['time_point'], model_data['price'],
                          legend_label=model_name, color=self.colors[i % len(self.colors)],
                          line_width=2)

            p.legend.location = "top_left"
            plots.append(p)

        return gridplot(plots, ncols=2)

    def create_occupancy_vs_price_plot(self, results_df: pd.DataFrame):
        """Create scatter plot showing occupancy vs price relationship"""
        p = figure(title='Occupancy Rate vs Price Relationship',
                  x_axis_label='Occupancy Rate', y_axis_label='Price ($)',
                  width=600, height=400)

        for i, model_name in enumerate(['linear', 'demand_based', 'competitive']):
            model_data = results_df[results_df['model'] == model_name]
            if not model_data.empty:
                p.scatter(model_data['occupancy_rate'], model_data['price'],
                         legend_label=model_name, color=self.colors[i],
                         alpha=0.6, size=5)

        p.legend.location = "top_left"
        return p

    def create_summary_statistics(self, results_df: pd.DataFrame) -> pd.DataFrame:
        """Create summary statistics for analysis"""
        summary = results_df.groupby(['model', 'space_id']).agg({
            'price': ['mean', 'min', 'max', 'std'],
            'occupancy_rate': 'mean'
        }).round(2)

        return summary


In [77]:
# STEP 5: MAIN EXECUTION AND DEMONSTRATION
# =============================================================================

def main_simulation():
    """Main function to run the complete simulation"""

    print("=== Dynamic Parking Pricing Simulation ===")
    print("Initializing simulator...")

    # Initialize simulator
    simulator = ParkingPricingSimulator(num_spaces=14, num_days=5)

    # Generate dataset
    dataset = simulator.generate_dataset()

    # Save dataset for reference
    dataset.to_csv('parking_dataset.csv', index=False)
    print("Dataset saved as 'parking_dataset.csv'")

    # Run real-time simulation
    results = simulator.simulate_real_time_pricing(dataset, delay_seconds=0.1)

    # Save results
    results.to_csv('pricing_results.csv', index=False)
    print("Results saved as 'pricing_results.csv'")

    # Create visualizations
    print("Creating visualizations...")
    visualizer = PricingVisualizer()

    # Enable Bokeh in notebook
    try:
        output_notebook()

        # Create and show plots
        price_plots = visualizer.create_price_comparison_plot(results)
        occupancy_plot = visualizer.create_occupancy_vs_price_plot(results)

        print("Displaying price comparison plots...")
        show(price_plots)

        print("Displaying occupancy vs price relationship...")
        show(occupancy_plot)

    except Exception as e:
        print(f"Visualization error: {e}")
        print("Please ensure Bokeh is properly installed and you're running in a Jupyter notebook")

    # Generate summary statistics
    summary = visualizer.create_summary_statistics(results)
    print("\n=== SUMMARY STATISTICS ===")
    print(summary)

    # Model performance analysis
    print("\n=== MODEL ANALYSIS ===")
    for model_name in ['linear', 'demand_based', 'competitive']:
        model_data = results[results['model'] == model_name]
        avg_price = model_data['price'].mean()
        price_volatility = model_data['price'].std()

        print(f"{model_name.title()} Model:")
        print(f"  Average Price: ${avg_price:.2f}")
        print(f"  Price Volatility: ${price_volatility:.2f}")
        print(f"  Price Range: ${model_data['price'].min():.2f} - ${model_data['price'].max():.2f}")

    return dataset, results

In [78]:
# STEP 6: PATHWAY INTEGRATION (REAL-TIME STREAMING)
# =============================================================================

class PathwayPricingEngine:
    """Pathway-based real-time pricing engine"""

    def __init__(self):
        self.linear_model = LinearPricingModel()
        self.demand_model = DemandBasedPricingModel()
        self.competitive_model = CompetitivePricingModel()

        # Store current prices for each space
        self.current_prices = {i: 10.0 for i in range(14)}

    def process_linear_pricing(self, occupancy: int, capacity: int,
                             space_id: int, **kwargs) -> float:
        """Process linear pricing in Pathway"""
        price = self.linear_model.calculate_price(
            occupancy=occupancy,
            capacity=capacity,
            previous_price=self.current_prices.get(space_id, 10.0)
        )
        self.current_prices[space_id] = price
        return price

    def process_demand_pricing(self, occupancy: int, capacity: int,
                             queue_length: int, traffic_level: int,
                             is_special_day: bool, vehicle_type: str,
                             **kwargs) -> float:
        """Process demand-based pricing in Pathway"""
        return self.demand_model.calculate_price(
            occupancy=occupancy,
            capacity=capacity,
            queue_length=queue_length,
            traffic_level=traffic_level,
            is_special_day=is_special_day,
            vehicle_type=vehicle_type
        )

    def process_competitive_pricing(self, space_id: int, occupancy: int,
                                  capacity: int, queue_length: int,
                                  traffic_level: int, is_special_day: bool,
                                  vehicle_type: str, latitude: float,
                                  longitude: float, **kwargs) -> float:
        """Process competitive pricing in Pathway"""
        # For simplicity, we'll use demand-based pricing as base
        # In a full implementation, you'd query other spaces' current prices
        return self.demand_model.calculate_price(
            occupancy=occupancy,
            capacity=capacity,
            queue_length=queue_length,
            traffic_level=traffic_level,
            is_special_day=is_special_day,
            vehicle_type=vehicle_type
        )

def setup_pathway_streaming(dataset_path: str = "parking_dataset.csv"):
    """Set up Pathway for real-time streaming simulation"""

    try:
        import pathway as pw

        print("Setting up Pathway streaming...")

        # Define input schema
        class ParkingInputSchema(pw.Schema):
            day: int
            time_point: int
            space_id: int
            latitude: float
            longitude: float
            capacity: int
            occupancy: int
            queue_length: int
            vehicle_type: str
            traffic_level: int
            is_special_day: bool

        # Create pricing engine
        pricing_engine = PathwayPricingEngine()

        # Set up data source with streaming simulation
        # This simulates real-time data ingestion
        data_source = pw.io.csv.read(
            dataset_path,
            schema=ParkingInputSchema,
            mode="streaming"  # Enable streaming mode
        )

        # Apply pricing logic using UDFs (User Defined Functions)
        def apply_linear_pricing(row):
            return pricing_engine.process_linear_pricing(
                occupancy=row.occupancy,
                capacity=row.capacity,
                space_id=row.space_id
            )

        def apply_demand_pricing(row):
            return pricing_engine.process_demand_pricing(
                occupancy=row.occupancy,
                capacity=row.capacity,
                queue_length=row.queue_length,
                traffic_level=row.traffic_level,
                is_special_day=row.is_special_day,
                vehicle_type=row.vehicle_type
            )

        def apply_competitive_pricing(row):
            return pricing_engine.process_competitive_pricing(
                space_id=row.space_id,
                occupancy=row.occupancy,
                capacity=row.capacity,
                queue_length=row.queue_length,
                traffic_level=row.traffic_level,
                is_special_day=row.is_special_day,
                vehicle_type=row.vehicle_type,
                latitude=row.latitude,
                longitude=row.longitude
            )

        # Create pricing computations
        pricing_results = data_source.select(
            pw.this.day,
            pw.this.time_point,
            pw.this.space_id,
            pw.this.occupancy,
            pw.this.capacity,
            pw.this.queue_length,
            pw.this.traffic_level,
            pw.this.is_special_day,
            pw.this.vehicle_type,
            occupancy_rate=pw.this.occupancy / pw.this.capacity,
            linear_price=pw.apply(apply_linear_pricing, pw.this),
            demand_price=pw.apply(apply_demand_pricing, pw.this),
            competitive_price=pw.apply(apply_competitive_pricing, pw.this)
        )

        # Add timestamp for real-time tracking
        pricing_results = pricing_results.with_columns(
            timestamp=pw.this.day * 18 + pw.this.time_point
        )

        # Write results to output
        pw.io.csv.write(pricing_results, "pathway_pricing_results.csv")

        # Optional: Write to JSON for real-time dashboard
        pw.io.jsonlines.write(pricing_results, "pathway_pricing_stream.jsonl")

        print("Pathway streaming setup complete!")
        print("Starting real-time processing...")

        # Run the streaming pipeline
        pw.run()

        return pricing_results

    except ImportError:
        print("Pathway not installed. Please install using: pip install pathway-python")
        return None
    except Exception as e:
        print(f"Pathway setup error: {e}")
        return None

def run_pathway_simulation():
    """Run the complete Pathway-based simulation"""

    print("=== Pathway Real-Time Streaming Simulation ===")

    # First, generate the dataset
    simulator = ParkingPricingSimulator(num_spaces=14, num_days=3)
    dataset = simulator.generate_dataset()
    dataset.to_csv('parking_dataset.csv', index=False)

    print("Dataset generated for Pathway streaming...")

    # Set up and run Pathway streaming
    pathway_results = setup_pathway_streaming('parking_dataset.csv')

    if pathway_results:
        print("Pathway simulation completed successfully!")
        print("Output files:")
        print("- pathway_pricing_results.csv: Streaming results")
        print("- pathway_pricing_stream.jsonl: Real-time stream data")
    else:
        print("Pathway simulation failed. Using alternative simulation...")
        # Fall back to regular simulation
        results = simulator.simulate_real_time_pricing(dataset, delay_seconds=0.1)
        results.to_csv('pricing_results.csv', index=False)
        print("Alternative simulation completed.")

    return pathway_results


In [79]:
# STEP 7: ENHANCED VISUALIZATION WITH REAL-TIME UPDATES
# =============================================================================

def create_real_time_dashboard():
    """Create a real-time dashboard for monitoring prices"""

    try:
        from bokeh.plotting import figure, curdoc
        from bokeh.models import ColumnDataSource
        from bokeh.layouts import column, row
        from bokeh.models.widgets import Div
        import json

        # Read streaming data
        try:
            with open('pathway_pricing_stream.jsonl', 'r') as f:
                stream_data = [json.loads(line) for line in f]
        except FileNotFoundError:
            # Use CSV data if JSONL not available
            stream_data = pd.read_csv('pricing_results.csv').to_dict('records')

        # Create data source for real-time updates
        source = ColumnDataSource(data=dict(
            time=[],
            space_id=[],
            linear_price=[],
            demand_price=[],
            competitive_price=[],
            occupancy_rate=[]
        ))

        # Create plots
        price_plot = figure(title="Real-Time Pricing Updates",
                          x_axis_label="Time", y_axis_label="Price ($)",
                          width=800, height=400)

        price_plot.line('time', 'linear_price', source=source,
                       legend_label="Linear Model", color='red', line_width=2)
        price_plot.line('time', 'demand_price', source=source,
                       legend_label="Demand Model", color='blue', line_width=2)
        price_plot.line('time', 'competitive_price', source=source,
                       legend_label="Competitive Model", color='green', line_width=2)

        # Occupancy plot
        occupancy_plot = figure(title="Occupancy Rates",
                              x_axis_label="Time", y_axis_label="Occupancy Rate",
                              width=800, height=300)

        occupancy_plot.line('time', 'occupancy_rate', source=source,
                          color='orange', line_width=2)

        # Status display
        status_div = Div(text="<h3>Real-Time Parking Pricing Dashboard</h3>")

        # Layout
        layout = column(status_div, price_plot, occupancy_plot)

        print("Real-time dashboard created successfully!")
        return layout

    except Exception as e:
        print(f"Dashboard creation error: {e}")
        return None

if __name__ == "__main__":
    # Choose simulation type
    use_pathway = True  # Set to True for Pathway streaming, False for regular simulation

    if use_pathway:
        print("Running Pathway-based real-time simulation...")
        pathway_results = run_pathway_simulation()

        # Create real-time dashboard
        dashboard = create_real_time_dashboard()
        if dashboard:
            print("Real-time dashboard available!")
    else:
        print("Running standard simulation...")
        dataset, results = main_simulation()

    print("\n=== SIMULATION COMPLETE ===")
    print("Files generated:")
    if use_pathway:
        print("- parking_dataset.csv: Raw parking data")
        print("- pathway_pricing_results.csv: Pathway streaming results")
        print("- pathway_pricing_stream.jsonl: Real-time stream data")
    else:
        print("- parking_dataset.csv: Raw parking data")
        print("- pricing_results.csv: Pricing simulation results")

    print("\nNext steps:")
    print("1. Analyze the results in the generated CSV files")
    print("2. Modify model parameters to test different pricing strategies")
    print("3. Experiment with Pathway real-time streaming features")
    print("4. Customize the real-time dashboard for your needs")

    # Installation guide
    print("\n=== INSTALLATION GUIDE ===")
    print("To run with Pathway in Google Colab:")
    print("1. !pip install pathway-python")
    print("2. !pip install bokeh")
    print("3. Set use_pathway = True in the main execution")
    print("4. Run the complete simulation")

Running Pathway-based real-time simulation...
=== Pathway Real-Time Streaming Simulation ===
Generating dataset...
Day 1/3 generated
Day 2/3 generated
Day 3/3 generated
Dataset generated: 756 records
Dataset generated for Pathway streaming...
Setting up Pathway streaming...
Pathway setup error: Function pathway.internals.common.apply() parameter args=<class 'pathway.internals.thisclass.this'> violates type hint typing.Union[pathway.internals.expression.ColumnExpression, NoneType, int, float, str, bytes, bool, pathway.engine.Pointer, datetime.datetime, datetime.timedelta, numpy.ndarray, pathway.internals.json.Json, dict[str, typing.Any], tuple[typing.Any, ...], pathway.engine.Error, pathway.engine.Pending], as <class "pathway.internals.thisclass.ThisMetaclass"> <class 'pathway.internals.thisclass.this'> not bytes, <protocol "pathway.engine.Pointer">, <protocol "numpy.ndarray">, <protocol "datetime.timedelta">, str, <protocol "pathway.internals.json.Json">, <class "datetime.datetime">, <